### --- Day 6: Guard Gallivant ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/6

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 6);
WriteLines(inputLines);

Loading puzzle file: Day6.txt
Total lines: 130
Max line length: 130

......#........#.............#.##..........................................#.......#...#........#......................#..........
.......................#...#................................#..#.........................................#.....#.....#............
.......................................#.................#..................#....#........................................#.......
.............#...................#.#...............................#....#....#.#.......................................#..#.......
...............................#........##.#..............................#..............................#..#...#..#..............


In [4]:
string[] testInputLines = [
    "....#.....",
    ".........#",
    "..........",
    "..#.......",
    ".......#..",
    "..........",
    ".#..^.....",
    "........#.",
    "#.........",
    "......#...",
];

In [5]:
IEnumerable<(Point point, Point direction)> Walk(CharGrid grid)
{
    var currentPoint = grid.Enumerate().Single(pch => pch.ch is '^').point;
    var currentDirection = Up;

    yield return (currentPoint, currentDirection);

    int safety = 0;
    const int safety_limit = 1_000_000;
    while (true)
    {
        if (safety++ > safety_limit) {
            throw new Exception("Exceeded safety limit");
        }

        var nextPoint = currentPoint + currentDirection;
        if (!grid.IsValid(nextPoint)) {
            // Exited the map, we're done
            break;
        }
        else if (grid[nextPoint] is '#') {
            // Hit an object, turn right
            currentDirection = TurnRight(currentDirection);
            continue;
        }
        
        // Free space ahead, take a step
        currentPoint = nextPoint;
        yield return (currentPoint, currentDirection);
    }
}

int DistinctPositions(string[] inputLines)
{
    CharGrid grid = new(inputLines);

    var visited = Walk(grid).Select(pointDir => pointDir.point).Distinct();
    return visited.Count();
}

In [6]:
// In this example, the guard will visit 41 distinct positions on your map.

var testAnswer = DistinctPositions(testInputLines);
Console.WriteLine(testAnswer);

41


In [7]:
// Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

var part1Answer = DistinctPositions(inputLines);
Console.WriteLine(part1Answer);

5080


In [8]:
// 5080 is correct!
Ensure(5080, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/6

In [10]:
// Retrospective note: I tried a few ideas with this one, but ultimately just
// brute-forced it by placing an obstruction along each point in the original path and
// re-walking it :)

In [11]:
using WalkHistory = SCG.HashSet<(Point point, Point direction)>;

int CountObstructionPoints(string[] inputLines)
{
    CharGrid grid = new(inputLines);

    var allSteps = Walk(grid).ToList();
    var allPoints = allSteps.Select(pointDir => pointDir.point).ToHashSet();

    // Don't want to place an obstruction directly on our starting position
    allPoints.Remove(allSteps[0].point); 

    return allPoints.Where(point => ContainsLoop(grid, point)).Count();
}

bool ContainsLoop(CharGrid originalGrid, Point obstructionPoint)
{
    var newGrid = originalGrid.Clone();
    newGrid[obstructionPoint] = '#';

    WalkHistory walkHistory = new();
    foreach (var pointDir in Walk(newGrid))
    {
        var visited = !walkHistory.Add(pointDir);
        if (visited) {
            // Found a loop
            return true;
        }
    }
    return false;
}

In [12]:
// ...in this example, there are 6 different positions you could choose.

var part2TestAnswer = CountObstructionPoints(testInputLines);
Console.WriteLine(part2TestAnswer);

6


In [13]:
// You need to get the guard stuck in a loop by adding a single new obstruction.
// How many different positions could you choose for this obstruction?

var part2Answer = CountObstructionPoints(inputLines);
Console.WriteLine(part2Answer);

1919


In [14]:
// 1919 is correct!
Ensure(1919, part2Answer);